In [1]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_datagen = ImageDataGenerator(
                                          
                                          rescale = 1./255,
                                          width_shift_range = 0.2,
                                          height_shift_range = 0.2,
                                          shear_range = 0.2,
                                          zoom_range = 0.2,
                                          horizontal_flip = True
                                   
                                          )

In [3]:
train = train_datagen.flow_from_directory('/content/drive/MyDrive/rohani_recog',
                                                      target_size = (224,224),
                                                      batch_size = 6,
                                                      class_mode = 'categorical',
                                                      subset = 'training'
    )

Found 157 images belonging to 2 classes.


In [4]:
model = tf.keras.models.Sequential([
         Conv2D(32, (3, 3), input_shape=(224,224,3), activation='relu'),
         MaxPool2D((2, 2)),
         Conv2D(64, (3, 3), activation='relu'),
         MaxPool2D((2, 2)),
         Conv2D(128, (3, 3), activation='relu'),
         MaxPool2D((2, 2)),
         Flatten(),
         Dense(512, activation='relu'),
         Dense(2, activation='softmax')                          
])

model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss=tf.keras.losses.binary_crossentropy,
              metrics='accuracy')

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [5]:
history = model.fit(train,  epochs=40)

Epoch 1/40
27/27 [==============================] - 86s 2s/step - loss: 0.7450 - accuracy: 0.5096
Epoch 2/40
27/27 [==============================] - 4s 155ms/step - loss: 0.6909 - accuracy: 0.5478
Epoch 3/40
27/27 [==============================] - 4s 153ms/step - loss: 0.6784 - accuracy: 0.5796
Epoch 4/40
27/27 [==============================] - 4s 154ms/step - loss: 0.6261 - accuracy: 0.6497
Epoch 5/40
27/27 [==============================] - 4s 154ms/step - loss: 0.6196 - accuracy: 0.6561
Epoch 6/40
27/27 [==============================] - 4s 151ms/step - loss: 0.5996 - accuracy: 0.6879
Epoch 7/40
27/27 [==============================] - 4s 151ms/step - loss: 0.5356 - accuracy: 0.7325
Epoch 8/40
27/27 [==============================] - 4s 151ms/step - loss: 0.5719 - accuracy: 0.7197
Epoch 9/40
27/27 [==============================] - 4s 149ms/step - loss: 0.5063 - accuracy: 0.7580
Epoch 10/40
27/27 [==============================] - 4s 146ms/step - loss: 0.5825 - accuracy: 0.7006
E

In [8]:
model.save('rohani.h5')

In [9]:
!pip install pyTelegramBotAPI

     |████████████████████████████████| 101 kB 6.2 MB/s 
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.0.1-py3-none-any.whl size=79552 sha256=bac0f5f651b745ba501440564046516921152aa7214e334bae47a4544ca5a5b6
  Stored in directory: /root/.cache/pip/wheels/2f/c4/5a/7440a72b0ee7aad2d7ba7b2b44a93953183a30b017f604bad3
Successfully built pyTelegramBotAPI


In [11]:
import cv2
from tensorflow.python.keras.models import load_model
import numpy as np
import telebot
from telebot import types

bot = telebot.TeleBot('1985737443:AAEbRJ8Nv3QrY-ACfYWBb0Hh4nL4_3u86NE')

@bot.message_handler(content_types=['photo'])
def photo(message):
    
    fileID = message.photo[-1].file_id
    print(fileID)
    
    file_info = bot.get_file(fileID)
    print(file_info)
    downloaded_file = bot.download_file(file_info.file_path)
    print(downloaded_file)
    with open(fileID, 'wb') as new_file:
        new_file.write(downloaded_file)

    

    image = cv2.imread(fileID)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    image = image / 255
    image = image.reshape(1, 224, 224, 3)

    pred = model.predict([image])

    result = np.argmax(pred)

    if result == 1:
        print('از روحانیت  می باشد')
        bot.reply_to(message, 'از روحانیت  می باشد')

    elif result == 0:
        print('فردی عادی است')
        bot.reply_to(message, 'فردی عادی است ')


@bot.message_handler(commands=['SendPic'])
def send_pic(message):
    bot.send_message(message.chat.id, 'tanx', reply_markup=buttons)


@bot.message_handler(commands=['start'])
def say_hello(message):
    bot.send_message(message.chat.id, f'خوش اومدین {message.from_user.first_name} ')


@bot.message_handler(func=lambda message: True)
def send_unknown(message):
    bot.reply_to(message, 'نامفهومه برام تا حالا ندیدمش!')


bot.polling()




AgACAgQAAxkBAAPzYUjICjryzM_Ra73l-NviL3VFXtcAAhS1MRv-w0lSYXc6nqCwUnkBAAMCAANtAAMgBA
{'file_id': 'AgACAgQAAxkBAAPzYUjICjryzM_Ra73l-NviL3VFXtcAAhS1MRv-w0lSYXc6nqCwUnkBAAMCAANtAAMgBA', 'file_unique_id': 'AQADFLUxG_7DSVJy', 'file_size': 5790, 'file_path': 'photos/file_12.jpg'}
b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00\x84\x00\t\x06\x07\x12\x12\x12\x15\x12\x12\x13\x15\x10\x15\x10\x10\x15\x15\x15\x15\x15\x15\x15\x15\x15\x15\x15\x17\x16\x17\x16\x16\x15\x15\x15\x18\x1d( \x18\x1a%\x1b\x15\x16!1!%)+...\x17\x1f383-7(-.+\x01\n\n\n\x0e\r\x0e\x1b\x10\x10\x18+\x1d\x1d\x1d--+-+------+----+--------------------+------7+---7\xff\xc0\x00\x11\x08\x00\xa8\x01,\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1c\x00\x00\x00\x07\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\xff\xc4\x00?\x10\x00\x02\x01\x02\x03\x05\x05\x04\x08\x05\x03\x05\x01\x00\x00\x00\x01\x02\x00\x03\x11\x04\x05!\x06\x121AQaq\x81\x91\xa1\x13"2\xc1\x07\x14#BRb